In [2]:
import diamond_music_utils as dmu
from importlib import reload
reload(dmu)
import numpy as np
import pprint
pp = pprint.PrettyPrinter(sort_dicts=False)
from fractions import Fraction
import copy
import logging
from numpy.random import default_rng
rng = np.random.default_rng()
CSD_FILE = 'ball3.csd'
LOGNAME = 'ball3.log'
WAV_OUT = '/home/prent/Music/sflib/ball3.wav'
CON_OUT = '/home/prent/Music/sflib/ball3-t'

In [3]:
csound_running = False

In [237]:
all_ratios = np.array(dmu.build_all_ratios())
ratio_strings = dmu.build_ratio_strings(all_ratios) # create a (16,16) array of ratios in string form. Change those that are '1' to '1/1'
print(f'{all_ratios.shape = }')
# print the ratios as fractions
limit = 31
end_denom = limit + 1
start_denom = (end_denom) // 2
o_numerator = np.arange(start_denom, end_denom, 1) # create a list of overtones
u_denominator = np.arange(start_denom, end_denom, 1) # create a list of undertones
print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
i = 0
for oton_root in u_denominator:
    print()
    for overtone in o_numerator:
        print(f'{str(Fraction(all_ratios[i]).limit_denominator())}', end = '\t')
        i += 1
all_ratio_strings = ratio_strings.reshape(256,)


all_ratios.shape = (256,)
Tonality Diamond in the Cassandra orientation to the 31-limit
1	17/16	9/8	19/16	5/4	21/16	11/8	23/16	3/2	25/16	13/8	27/16	7/4	29/16	15/8	31/16	
32/17	1	18/17	19/17	20/17	21/17	22/17	23/17	24/17	25/17	26/17	27/17	28/17	29/17	30/17	31/17	
16/9	17/9	1	19/18	10/9	7/6	11/9	23/18	4/3	25/18	13/9	3/2	14/9	29/18	5/3	31/18	
32/19	34/19	36/19	1	20/19	21/19	22/19	23/19	24/19	25/19	26/19	27/19	28/19	29/19	30/19	31/19	
8/5	17/10	9/5	19/10	1	21/20	11/10	23/20	6/5	5/4	13/10	27/20	7/5	29/20	3/2	31/20	
32/21	34/21	12/7	38/21	40/21	1	22/21	23/21	8/7	25/21	26/21	9/7	4/3	29/21	10/7	31/21	
16/11	17/11	18/11	19/11	20/11	21/11	1	23/22	12/11	25/22	13/11	27/22	14/11	29/22	15/11	31/22	
32/23	34/23	36/23	38/23	40/23	42/23	44/23	1	24/23	25/23	26/23	27/23	28/23	29/23	30/23	31/23	
4/3	17/12	3/2	19/12	5/3	7/4	11/6	23/12	1	25/24	13/12	9/8	7/6	29/24	5/4	31/24	
32/25	34/25	36/25	38/25	8/5	42/25	44/25	46/25	48/25	1	26/25	27/25	28/25	29/25	6/5	31/25	
16/13	17/13	18/13	19/13	20/13	21/13	22/13	23/1

In [5]:
# print the ratios as strings
print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
for oton_root in np.arange(16):
    print()
    for overtone in np.arange(16):
        print(f'{ratio_strings[overtone, oton_root]}', end = '\t')

Tonality Diamond in the Cassandra orientation to the 31-limit
1/1	32/17	16/9	32/19	8/5	32/21	16/11	32/23	4/3	32/25	16/13	32/27	8/7	32/29	16/15	32/31	
17/16	1/1	17/9	34/19	17/10	34/21	17/11	34/23	17/12	34/25	17/13	34/27	17/14	34/29	17/15	34/31	
9/8	18/17	1/1	36/19	9/5	12/7	18/11	36/23	3/2	36/25	18/13	4/3	9/7	36/29	6/5	36/31	
19/16	19/17	19/18	1/1	19/10	38/21	19/11	38/23	19/12	38/25	19/13	38/27	19/14	38/29	19/15	38/31	
5/4	20/17	10/9	20/19	1/1	40/21	20/11	40/23	5/3	8/5	20/13	40/27	10/7	40/29	4/3	40/31	
21/16	21/17	7/6	21/19	21/20	1/1	21/11	42/23	7/4	42/25	21/13	14/9	3/2	42/29	7/5	42/31	
11/8	22/17	11/9	22/19	11/10	22/21	1/1	44/23	11/6	44/25	22/13	44/27	11/7	44/29	22/15	44/31	
23/16	23/17	23/18	23/19	23/20	23/21	23/22	1/1	23/12	46/25	23/13	46/27	23/14	46/29	23/15	46/31	
3/2	24/17	4/3	24/19	6/5	8/7	12/11	24/23	1/1	48/25	24/13	16/9	12/7	48/29	8/5	48/31	
25/16	25/17	25/18	25/19	5/4	25/21	25/22	25/23	25/24	1/1	25/13	50/27	25/14	50/29	5/3	50/31	
13/8	26/17	13/9	26/19	13/10	26/21	13/11	26/23	13

## Make a multivoice piece
This is the first attempt at composing a multivoice piece where every note is written out. Later I can automate the process of choosing variables

In [35]:
#
# Start up an instance of csound
#
#
def start_csound():
    global csound_running
    if not csound_running: 
        dmu.start_logger(LOGNAME)
        csd_content, lines = dmu.load_csd(CSD_FILE)
        logging.info(f'Starting to log to {LOGNAME}')
        logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
        cs, pt = dmu.load_csound(csd_content)
        csound_running = True
        return(cs, pt)
    else: 
        print(f'csound is already running')
        return(None, None)
#
#
# stop the csound instance
#
#
def stop_csound(cs, pt):
    global csound_running
    if csound_running:
        dmu.printMessages(cs)
        pt.stop() # this is important I think. It closes the output file.
        pt.join()   
        dmu.printMessages(cs)    
        cs.reset()
        csound_running = False
    else:
        print(f"csound isn't running")

In [36]:
# Is there anything else I need to track about the voice? If not, this could be simplified
# track the name, the start time, and the instrument number
voice_time = {'fpn': {'full': 'finger piano', 'start': 0, 'number': 1},
      'bnp': {'full': 'bass finger piano', 'start': 0, 'number': 2},
      'bdl': {'full': 'bass balloon drum', 'start': 0, 'number': 3},
      'bdm': {'full': 'medium balloon drum', 'start': 0, 'number': 4},
      'bdh': {'full': 'high balloon drum', 'start': 0, 'number': 5},
      'bfl': {'full': 'bass flute', 'start': 0, 'number': 6},
      'obo': {'full': 'oboe', 'start': 0, 'number': 7},
      'cla': {'full': 'clarinet', 'start': 0, 'number': 8},
      'bss': {'full': 'bassoon', 'start': 0, 'number': 9},
      'frn': {'full': 'french horn', 'start': 0, 'number': 10}}

pp.pprint(voice_time)
print(f'{voice_time["fpn"]["full"] = }')
print(f'{voice_time["fpn"]["start"] = }')
print(f'{voice_time["fpn"]["number"] = }')

{'fpn': {'full': 'finger piano', 'start': 0, 'number': 1},
 'bnp': {'full': 'bass finger piano', 'start': 0, 'number': 2},
 'bdl': {'full': 'bass balloon drum', 'start': 0, 'number': 3},
 'bdm': {'full': 'medium balloon drum', 'start': 0, 'number': 4},
 'bdh': {'full': 'high balloon drum', 'start': 0, 'number': 5},
 'bfl': {'full': 'bass flute', 'start': 0, 'number': 6},
 'obo': {'full': 'oboe', 'start': 0, 'number': 7},
 'cla': {'full': 'clarinet', 'start': 0, 'number': 8},
 'bss': {'full': 'bassoon', 'start': 0, 'number': 9},
 'frn': {'full': 'french horn', 'start': 0, 'number': 10}}
voice_time["fpn"]["full"] = 'finger piano'
voice_time["fpn"]["start"] = 0
voice_time["fpn"]["number"] = 1


In [37]:
def init_voice_time_start():
    for short_name in voice_time:
        voice_time[short_name]["start"] = 0
    return 0

print(f'{[voice_time[short_name]["start"] for short_name in voice_time.keys()]}')
result = init_voice_time_start()
print(f'{result = }')
print(f'{[voice_time[short_name]["start"] for short_name in voice_time.keys()]}')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
result = 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
# best combination of rank and inversion to minimize the distances between notes in a chord for ranks A & B
# these were found by systematically testing every possible combination and finding those that were closer than 0.7 and 1.3 ratio from each other
best_rank_inversion_combos = np.array([["A", "A", 1, 2],["A", "A", 1, 4],["A", "A", 2, 1],["A", "A", 2, 3],["A", "A", 3, 2],
    ["A", "A", 3, 4],["A", "A", 4, 1],["A", "A", 4, 3],["A", "B", 1, 1],["A", "B", 1, 4],["A", "B", 2, 1],["A", "B", 2, 2],
    ["A", "B", 3, 2],["A", "B", 3, 3],["A", "B", 4, 3],["A", "B", 4, 4],["B", "A", 1, 1],["B", "A", 1, 2],["B", "A", 2, 2],
    ["B", "A", 2, 3],["B", "A", 3, 3],["B", "A", 3, 4],["B", "A", 4, 1],["B", "A", 4, 4],["B", "B", 1, 2],["B", "B", 1, 4],
    ["B", "B", 2, 1],["B", "B", 2, 3],["B", "B", 3, 2],["B", "B", 3, 4],["B", "B", 4, 1],["B", "B", 4, 3],])

# combination of trills_8_step and envelope to convert an 8 step trill into fewer steps
# envelope #15: 1 up, 1 down, 1 up is 1/6 of the hold value
# 14: 2 down & up 1/4 of the hold value
# 13: 4 down & up 1/2 of the hold value
# 12: 5 down & up 2/3 of the hold value
# 11: 6 down & up 3/4 of the hold value
# 1:  8 down & up full hold value
# 2: exponential decay for a short chop
# 8: fast decay for bass finger piano punch
combo_trills = np.array([[15,4, 20], [14, 4, 16], [13, 4, 8],[12, 4, 6],[11, 4, 5],[1, 4, 4]])

In [39]:
# select a random combination of ranks and intervals and return them as an array
def choose_best_rank_inversion_combos():
    return(best_rank_inversion_combos[rng.integers(low = 0, high = best_rank_inversion_combos.shape[0])])
def choose_combo_trills():
    return(combo_trills[rng.integers(low = 0, high = combo_trills.shape[0])])
def choose_valid_rank():
    # return a valid rank
    return rng.choice([rank for rank in dmu.show_inversions().keys()])
def choose_valid_mode(rank):
    return(rng.choice([mode for mode in dmu.show_scales()[rank].keys()]))

In [40]:
# this makes one note with all 15 features - it also updates voice_time with new "start" values
def instrument(tone, octave, duration, hold, gliss, voice_name, \
               velocity = 60, stereo = 8, env =  1, \
               upsample = 1, renv = 1, gliss2 = 0, gliss3 = 0, volume = 30):
    
    voice = voice_time[voice_name]["number"] # this is the instrument number passed to csound
    
    # load an array with everything csound needs to create a note - the instrument number expected by the csound file has to be 1
    next_note = np.array((1, duration, hold, velocity, tone, octave, voice, stereo, \
                          env, gliss, upsample, renv, gliss2, gliss3, volume))
    
    # Before you return the note:
    # hold affects how long the instrument plays, duration is how much time until the next note in this voice plays
    # start time is calculated in this function by setting it to the "start" time for this voice plus the note's duration value
    
    current_time = voice_time[voice_name]["start"] # what is this voice's current start time
    
    voice_time[voice_name]["start"] += next_note[1] # add duration to the current voice's start time
    next_note[1] = current_time # overwrite column 1 (duration) with the new start_time
    return next_note

In [41]:
# pass a set of parameters and get back an array of notes
def chord_slide(note_dict):
    # Build a set of notes 
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], note_dict["combo"][0], int(note_dict["combo"][2])) # get four notes 
    tones_2 = dmu.build_chords(note_dict["mode"], note_dict["root"], note_dict["combo"][1], int(note_dict["combo"][3])) # get another four notes 
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask
    print(f'{note_dict["octv"] = }')
    # build_slides returns an ftable number for each note in the chord and updates global stored_gliss table
    
    gliss = dmu.build_slides(tones_1, tones_2, gliss_type = note_dict["gliss_type"]) 
    # The number of notes may not be the same as the number of instruments. We may need to duplicate some notes to reach the number of instruments
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    while num_notes < num_inst:
        # print(f'{num_inst = }, {num_notes = }, {tones_1.shape = }')
        # copy the duplicate the first element (0) to be the last element until the number of notes = number of instruments
        tones_1 = np.append(tones_1, tones_1[0], axis = None)
        octave = np.append(octave, octave[0], axis = None)
        gliss = np.append(gliss, gliss[0], axis = None)
        num_notes = tones_1.shape[0]
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
    
    dur = np.ones((num_inst,), dtype = float) * note_dict["dur"] # all the notes have the same duration
    # print(f'in chord_slides. {note_dict["hold"] = }')
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them
    # print(f'in chord_slides. {hold = }')
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    
    instruments = np.empty((num_inst * note_dict["repeats"], 15), dtype = float)
    for repeats in np.arange((note_dict["repeats"])):
        # repeat
        for inx in np.arange(num_inst):
            # spread the notes across the different instruments
            # instruments[:,inx] = instrument(tones_1[inx], octave[inx], dur[inx], hold[inx], gliss[inx], \
            #                     note_dict["instruments"][inx], env = env[inx], renv = renv[inx])
            
            instruments[repeats * num_inst + inx] = instrument(tones_1[inx], octave[inx], dur[inx], hold[inx],  gliss[inx], \
                                    note_dict["instruments"][inx], env = env[inx], renv = renv[inx])
            # print(f'{hold[inx] = }, {gliss[inx] = }')
            
    print(f'{instruments.shape = }')
    return instruments

In [103]:
# pass a set of parameters and get back an array of notes
def triad_arpeggio(note_dict):
    if note_dict["delay"] < 1: 
        print(f'In triad_arpeggio. {note_dict["delay"] = }. Minimum accepted is 1')
        return None
              
    # Build a set of notes                mode               root          random A or B              and random inversion
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], rng.choice(["A","B"]), rng.choice([1,2,3,4])) # get four notes 
    tones_1 = dmu.build_scales(note_dict["mode"], note_dict["root"], rng.choice(["A","B"])) # get 8 notes 
    tones_1 = np.tile(tones_1, note_dict["repeats"])
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask
    
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    
    dur = note_dict["dur"] # all the notes have the same duration
    hold = note_dict["hold"] # how long to hold them
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    #                                          4 notes 5 instruments 3 repeats
    print(f'creating empty instruments array. {num_notes = }, {num_inst = }')
    
    instruments = np.zeros((num_notes * num_inst, 15), dtype = float)
    print(f'{instruments.shape = }') # 20 slots

    # for repeats in np.arange(note_dict["repeats"]): 
    #     print(f'{repeats = }')
    roll = rng.choice([2,0,-2])
    iterations = 0

    for v_num, voice_string in zip(np.arange(num_inst), note_dict["instruments"]): # each instrument plays
        # print(f'{v_num = }, {voice_string = }')
        tones_1 = np.roll(tones_1, roll)
        # we need to update the start time for each instrument to account for the delay introduced between each instrument
        voice_time[note_dict["instruments"][v_num]]["start"] += dur[v_num] * note_dict["delay"] * v_num

        for note, octv in zip(tones_1, octave):
            # print(f'{note = }, {octv = }')
            instruments[iterations] = instrument(note, octv, dur[v_num], hold[v_num],\
                                         799, voice_string, env = env[v_num], renv = renv[v_num])
            iterations += 1


    print(f'{iterations = }') # 20.
    return instruments

In [104]:
# pass a set of parameters and get back an array of notes
def chord_chop(note_dict):
    # Build a set of notes                mode               root                 A or B                and inversion
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], rng.choice(["A","B"]), rng.choice([1,2,3,4])) # get four notes 
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask
    # The number of notes may not be the same as the number of instruments. We may need to duplicate some notes to reach the number of instruments
    # num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    while num_notes < num_inst:
        # print(f'{num_inst = }, {num_notes = }, {tones_1.shape = }')
        # copy the duplicate the first element (0) to be the last element until the number of notes = number of instruments
        tones_1 = np.append(tones_1, tones_1[0], axis = None)
        octave = np.append(octave, octave[0], axis = None)
        num_notes = tones_1.shape[0]
    if num_inst < num_notes:
        print(f'Not enough instruments specified in {note_dict["instruments"] = }, {num_notes = }')
        return None
    
    dur = np.ones((num_inst,), dtype = float) * note_dict["dur"] # all the notes have the same duration
    hold = np.ones((num_inst,), dtype = float) * note_dict["hold"] # how long to hold them
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for all of them
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # what should the right envelope be
    instruments = np.empty((num_inst * note_dict["repeats"], 15), dtype = float)
    print(f'{instruments.shape = }')
    
    for repeats in np.arange((note_dict["repeats"])):
        for inx in np.arange(num_inst):
            # spread the notes across the different instruments
            instruments[repeats * num_inst + inx] = instrument(tones_1[inx], octave[inx], dur[inx], hold[inx], 799, \
                                    note_dict["instruments"][inx], env = env[inx], renv = renv[inx])
            
    print(f'{instruments.shape = }')
    return instruments

In [496]:
# pass a set of parameters and get back an array of notes
def simple_hexad_arpeggio(note_dict):
    # Build a set of notes                mode               root         rank           and random inversion
    tones_1 = dmu.build_chords(note_dict["mode"], note_dict["root"], note_dict["rank"], rng.choice([1,2,3,4])) # get four notes 
   
    octave = dmu.build_scale_mask(tones_1) + note_dict["octv"] # get the octave mask and add it to the passed octave
    
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    dur = note_dict["dur"]   # each instrument can have it's own duration, the same for all repeats. But things get complicated if they don't
    hold = note_dict["hold"] # each instrument can have it's own hold value, repeated each repeat
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # same envelope for all of the instruments
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # same envelope for all of the instruments
    instruments = np.empty((num_notes * num_inst * note_dict["repeats"], 15), dtype = float)
    mix_mask = rng.choice(2, size=(instruments.shape[0]), p = [1 - note_dict["density"], note_dict["density"]])
    # print(f'{np.sum(mix_mask) = }')
    # print(f'{mix_mask.shape = }')
    iterations = 0
    for repeats in np.arange(note_dict["repeats"]): 
        roll = rng.choice([2, 0, -2])
        tones_1 = np.roll(tones_1, roll)
        for (note, octv) in zip(tones_1,octave): # iterate a match between a note and it's octave
            for inst in np.arange(num_inst):
                instruments[iterations] = instrument(note, octv, dur[inst], hold[inst] * mix_mask[iterations], \
                                        799, note_dict["instruments"][inst], env = env[inst], renv = env[inst])
                iterations += 1
    return instruments

In [561]:
# pass a set of parameters and get back an array of notes
def scale_arpeggio(note_dict):
              
    # set of notes from a scale based on  mode            root             rank 
    tones_1 = dmu.build_scales(note_dict["mode"], note_dict["root"], note_dict["rank"]) # get 8 notes 
    tones_1 = np.append(tones_1, tones_1[0])
    if note_dict["flip"]:
        print(f'before flip {tones_1 = }')
        tones_1 = np.flip(tones_1)
        print(f'after flip {tones_1 = }')
        
    octave = dmu.build_scale_mask(tones_1) # get the mask for this 8 tone scale
    print(f'{tones_1.shape = }, {tones_1 = }')
    print(f'{octave.shape = }, {octave = }')
    
    # tile with roll
    # print(f'{note_dict["repeats"] = }, {note_dict["roll"] = }')
    roll_tones = copy.deepcopy(tones_1)
    for repeat in np.arange(note_dict["repeats"]):
        roll_tones = np.roll(roll_tones, note_dict["roll"])
        if note_dict["flip"]: octave = np.append(octave, (dmu.build_scale_mask(roll_tones) - (repeat ))) 
        else: octave = np.append(octave, (dmu.build_scale_mask(roll_tones) + repeat + 1))  # get the octave mask of the scale plus 1
        tones_1 = np.append(tones_1, roll_tones)
        print(f'after appending a roll_tones: {repeat = }, {tones_1.shape = }')
    
    print(f'{tones_1.shape = }, {tones_1 = }')
    print(f'{octave.shape = }, {octave = }')
    octave += note_dict["octv"]
   
    num_inst = note_dict["instruments"].shape[0]
    num_notes = tones_1.shape[0]
    
    dur = note_dict["dur"] # each instrument has its own duration
    hold = note_dict["hold"] # each instrument has its own hold value
    env = np.ones((num_inst,), dtype = float) * note_dict["env"] # what envelope to use for instruments
    renv = np.ones((num_inst,), dtype = float) * note_dict["renv"] # right envelope 
    instruments = np.empty((num_notes * num_inst, 15), dtype = float)
    
    mix_mask = rng.choice(2, size=(instruments.shape[0]), p = [1 - note_dict["density"], note_dict["density"]])
    
    iterations = 0
    tones_1 = np.roll(tones_1, note_dict["roll"])
    for (note, octv) in zip(tones_1,octave): # iterate a match between a note and it's octave
        for inst in np.arange(num_inst):
            instruments[iterations] = instrument(note, octv, dur[inst], hold[inst] * mix_mask[iterations], \
                                    799, note_dict["instruments"][inst], env = env[inst], renv = renv[inst])
            iterations += 1
            
    return instruments

In [632]:
# reload(dmu)
init_voice_time_start()
stored_gliss = dmu.init_instrument() # resets the global glissando array and the global current_gliss_table variable to 800
winds = np.empty((0,15), dtype = float)
# reload(dmu)
woodwinds = np.array(["obo","cla","bfl","bss","frn"])
# This is a dictionary of note sequence generators, including a function to call, and all the values that that function requires
note_dict = {0: {'exec': chord_slide, 'mode': 'oton', 'root': '16/9', 'repeats': 1, 'combo': choose_best_rank_inversion_combos(),
                 'dur': combo_trills[0,1]/8, 'hold': combo_trills[0,2] / 8, 'octv': rng.choice([3, 4]), 
                 'gliss_type':'trill_8_step', 'env': combo_trills[0,0], 'renv': combo_trills[0,0], 'instruments': woodwinds},
             1: {'exec': chord_slide, 'mode': 'oton', 'root': '16/9', 'combo': choose_best_rank_inversion_combos(), 
                 'dur': combo_trills[1,1], 'hold': combo_trills[1,2], 'octv': 3, 'gliss_type':'trill_8_step', 'env': combo_trills[1,0], 
                 'renv': combo_trills[1,0], 'instruments': woodwinds},
             2: {'exec': chord_slide, 'mode': 'oton', 'root': '16/9', 'combo': choose_best_rank_inversion_combos(), 'dur': combo_trills[2,1], 
                 'hold': combo_trills[2,2], 'octv': 3, 'gliss_type':'trill_8_step', 'env': combo_trills[2,0], 
                 'renv': combo_trills[2,0], 'instruments': woodwinds},
             3: {'exec': chord_slide, 'mode': 'oton', 'root': '16/9', 'combo': choose_best_rank_inversion_combos(), 'dur': combo_trills[3,1], 
                 'hold': combo_trills[3,2], 'octv': 3, 'gliss_type':'trill_8_step', 'env': combo_trills[3,0], 
                 'renv': combo_trills[3,0], 'instruments': woodwinds},
             4: {'exec': chord_slide, 'mode': 'oton', 'root': '16/9', 'combo': choose_best_rank_inversion_combos(), 'dur': combo_trills[4,1], 
                 'hold': combo_trills[4,2], 'octv': 3, 'gliss_type':'trill_8_step', 'env': combo_trills[4,0], 
                 'renv': combo_trills[4,0], 'instruments': woodwinds},
             5: {'exec': chord_chop, 'mode': 'oton', 'root': '16/9', 'repeats': 3, 'dur': .15, 'hold': .45, 'octv': rng.choice([3, 4]), 
                 'env': 2, 'renv': 2, 'instruments': woodwinds},
             6: {'exec': simple_hexad_arpeggio, 'mode': 'oton', 'root': '16/9', 'rank': rng.choice(['A','B']), 'repeats': 25, 
                 'dur': np.array([.15, .15, .15, .15, .15]), 'hold': np.array([.14, .14, .14, .14, .14]), 
                 'octv': rng.choice([3, 4]), 'env': 8, 'renv': 8, 'instruments': woodwinds, 'density': 0.4 }, 
             7: {'exec': triad_arpeggio, 'mode': 'oton', 'root': '16/9', 'rank': rng.choice(['A','B']), 'repeats': 8,
                 'dur': np.array([.15, .15, .15, .15, .15]), 'hold': np.array([.14, .14, .14, .14, .14]), 
                 'octv': rng.choice([3, 4]), 'env': 8, 'renv': 8, 'instruments': woodwinds, 'delay': 6 }, 
             8: {'exec': scale_arpeggio, 'mode': 'oton', 'root': '16/9', 'rank': rng.choice(['A','B']), 'repeats': 2, 
                 'octave_offset': True,
                 'dur': np.array([.15, .15, .15, .15, .15]), 'hold': np.array([.14, .14, .14, .14, .14]), 'flip': False,
                 'octv': rng.choice([4, 3]), 'env': 1, 'renv': 1, 'instruments': woodwinds, 'roll': -2, 'density': 0.6},   
            }
for inx in np.arange(1):
    ind = 8              #  +-- existing winds array
    #                       |      +-- the new array created by calling note_dict with the exec value
    #                       |      |                      +-- argument passed to the specified function
    winds = np.concatenate((winds, note_dict[ind]["exec"](note_dict[ind])),axis = 0)
print(f'{winds.shape = }')

tones_1.shape = (9,), tones_1 = array([34, 36, 38, 40, 42, 44, 46, 32, 34])
octave.shape = (9,), octave = array([0, 0, 0, 0, 0, 0, 0, 0, 1])
after appending a roll_tones: repeat = 0, tones_1.shape = (18,)
after appending a roll_tones: repeat = 1, tones_1.shape = (27,)
tones_1.shape = (27,), tones_1 = array([34, 36, 38, 40, 42, 44, 46, 32, 34, 38, 40, 42, 44, 46, 32, 34, 34,
       36, 42, 44, 46, 32, 34, 34, 36, 38, 40])
octave.shape = (27,), octave = array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2,
       3, 2, 2, 2, 2])
winds.shape = (135, 15)


In [630]:
a, b = start_csound()
if a is not None:
    cs = a
    pt = b

csound is already running


In [631]:
print(f'glide ratios for all the slides in the whole piece:')
for row in dmu.retrieve_gliss_tables(): # pass some f tables for glissandi to the instance of csound
    pt.scoreEvent(0, 'f', row)
    print(f'{round(row[8],3)}')
print(f'here are the notes sent to csound')
print(f'InsSta Dur VelTon OctVoiSteEn1 Gls UpsRen2gl 3glVol')
for row in winds: # send the note information to csound
    pt.scoreEvent(0, 'i', row)
    print(f'{[int(row[item]) for item in np.arange(15)]}', end = '\t') # print the note values
    print(f'start: {round(row[1],3)},\tdur: {round(row[2],3)}')

glide ratios for all the slides in the whole piece:
here are the notes sent to csound
InsSta Dur VelTon OctVoiSteEn1 Gls UpsRen2gl 3glVol
[1, 0, 0, 60, 38, 4, 7, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.0,	dur: 0.14
[1, 0, 0, 60, 38, 4, 8, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.0,	dur: 0.14
[1, 0, 0, 60, 38, 4, 6, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.0,	dur: 0.14
[1, 0, 0, 60, 38, 4, 9, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.0,	dur: 0.0
[1, 0, 0, 60, 38, 4, 10, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.0,	dur: 0.14
[1, 0, 0, 60, 40, 4, 7, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.15,	dur: 0.0
[1, 0, 0, 60, 40, 4, 8, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.15,	dur: 0.14
[1, 0, 0, 60, 40, 4, 6, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.15,	dur: 0.0
[1, 0, 0, 60, 40, 4, 9, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.15,	dur: 0.0
[1, 0, 0, 60, 40, 4, 10, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.15,	dur: 0.14
[1, 0, 0, 60, 42, 4, 7, 8, 1, 799, 1, 1, 0, 0, 30]	start: 0.3,	dur: 0.0
[1, 0, 0, 60, 42, 4, 8, 8, 1, 799, 1, 1, 0, 0, 30]	start:

In [596]:
stop_csound(cs, pt)

0dBFS level = 32768.0
--Csound version 6.16 (double samples) Aug 10 2021
[commit: none]
libsndfile-1.0.31


In [532]:
60*60*24


86400